# Relax Bulk with CP2K
Use CP2K PBE to relax the lattice parameters of a unit cell

In [ ]:
from ase.filters import FrechetCellFilter
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from ase.db import connect
from ase import units, Atoms
from pathlib import Path
from co2rr.emery import generate_initial_structure, load_emery_dataset
from co2rr.cp2k import make_calculator
import pandas as pd
import numpy as np

Configuration

In [ ]:
out_dir = Path('relaxed-bulk')
skip_elems = []

## Load the Emery Dataset
The dataset with unstable and actinides filtered out

In [ ]:
emery = load_emery_dataset()

In [ ]:
emery.iloc[0]

In [ ]:
emery[['Chemical formula', 'In literature', 'Valence A', 'Valence B', 'Stability [eV/atom]']]

## Run the optimization
Make ASE calculators with our target settings then relax just the lattice parameter.

In [ ]:
for _, row in emery.iterrows():
    # Check if it's already done
    name=row['Chemical formula']
    with connect('cp2k-relax.db') as db:
        if db.count(f'name={name} and supercell={args.supercell_size}') > 0:
            continue

    # Skip bad elements
    if row['A'] in skip_elems or row['B'] in skip_elems:
        continue

    # Run, if needed
    atoms = generate_initial_structure(row)
    with make_calculator(atoms, cutoff=600, max_scf=500) as calc:
        # Delete the old run
        for f in ['cp2k.out', 'relax.log']:
            Path(f'run/{f}').write_text("")  # Clear it

        # Start by computing the stresses acting on the cell
        calc.directory = 'run'
        atoms.calc = calc
        stresses = atoms.get_stress()
        print(f'{name} - Initial volume: {atoms.get_volume()}. Stress: {stresses[:3].sum() / 3:.2f}')
        
        # Run the optimization
        atoms.set_constraint(FixAtoms(mask=[True] * len(atoms)))
        opt_atoms = FrechetCellFilter(atoms, hydrostatic_strain=True)  # Allow ASE to optimize the lattice parameter
        opt = BFGS(opt_atoms, trajectory='run/relax.traj', logfile='run/relax.log')
        opt.run(fmax=0.1)
        stresses = atoms.get_stress()
        print(f'{name} - Final volume: {atoms.get_volume()}. Stress: {stresses[:3].sum() / 3:.2f}')

        atoms.set_constraint()

    # Write to the output
    with connect('cp2k-relax.db') as db:
        try:
            a_val, b_val= int(row['Valence A']), int(row['Valence B'])
        except ValueError:
            a_val = b_val = -1
        db.write(atoms, name=name, a=row['A'], b=row['B'], a_val=a_val, b_val=b_val)